# LLM Iteration 25

**Aim**

To create an Auditor class that periodically collects information on the number of patients waiting and bed waiting times to be used for warm-up analysis.

**Prompt**

The admit_patient function should have the following structure:
```python
def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))

            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())
            
            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)

            length_of_stay_params = self.\
                update_length_of_stay(patient.patient_type, # destination
                                      route_choice)

            mu, sigma = calculate_mu_sigma(length_of_stay_params["mean"],
                                           length_of_stay_params["std_dev"])

            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
     
            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')
```

Update the model by creating a WarmupAuditor class. This should periodically collect information on the number of patients waiting for a bed and waiting times at specified times intervals. The class should be able to run the StrokeWard model and return a summary dataframe of the mean number of patients waiting for a bed and mean waiting times at each audit period. 

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**Additional code**

In [1]:
def trace(msg):
    '''
    Utility function for printing simulation.
    Set the TRACE constant to FALSE to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [2]:
TRACE = False

**LLM Output**

To update the model with the provided structure for the admit_patient function and create a WarmupAuditor class that periodically collects information on the number of patients waiting for a bed and waiting times at specified time intervals, you can modify the code as follows:

In [3]:
import simpy
import numpy as np
import math
import pandas as pd

In [4]:
class Scenario:
    # ... (Scenario class remains unchanged)

    ##### MODIFICATION - added Scenario class from previous iters #####
    def __init__(self, num_beds, run_length=60*2, seed=None):
        self.num_beds = num_beds
        self.run_length = run_length
        self.seed = seed
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self, num_runs):
        interarrival_distributions = {}
        seeds = self.generate_seeds(num_runs)
        for patient_type, seed in seeds.items():
            if seed[0] is not None:
                np.random.seed(seed)
                interarrival_distributions[patient_type] = np.random.\
                    default_rng(seed)
            else:
                interarrival_distributions[patient_type] = np.random.\
                    default_rng()
        return interarrival_distributions

    def generate_seeds(self, num_runs):
        seeds = {}
        for patient_type, seed in self.random_seeds.items():
            if self.seed is not None:
                seeds[patient_type] = [seed + i + self.seed for i in \
                                       range(num_runs)]
            else:
                seeds[patient_type] = [None for i in range(num_runs)]
        return seeds

    ###################################################################

In [5]:
class StrokeWard:
    def __init__(self, scenario):
        self.env = simpy.Environment()
        self.beds = simpy.Resource(self.env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.\
            create_distributions(num_runs=1)
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.patient_type_store = []
        self.los_store = []

    def update_length_of_stay(self, patient_type, destination):
        if patient_type == 'Stroke':
            if destination == 'ESD':
                return {"mean": 4.6, "std_dev": 4.8}
            elif destination == 'Other':
                # Mortality rate for Stroke patients going to Other
                if np.random.rand() < 0.02:
                     # Adjusted parameters for mortality case
                    return {"mean": 7.0, "std_dev": 8.7}
        return self.length_of_stay[patient_type]

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))

            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())

            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)

            length_of_stay_params = self.update_length_of_stay(patient.\
                                                               patient_type,
                                                               route_choice)

            mu, sigma = calculate_mu_sigma(length_of_stay_params["mean"],
                                           length_of_stay_params["std_dev"])

            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))

            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].exponential(
                        scenario.default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)

        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }

        summary_df = pd.DataFrame(summary_data)

        print("Summary:")
        print(summary_df)

In [6]:
class WarmupAuditor:
    def __init__(self, scenario, audit_interval):
        self.scenario = scenario
        self.audit_interval = audit_interval
        self.audit_times = []
        self.waiting_patients_audit = []
        self.wait_times_audit = []

    def run_model(self):
        stroke_ward = StrokeWard(self.scenario)

        PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
        for patient_type in PATIENTS:
            stroke_ward.env.process(stroke_ward.patient_arrival(patient_type))

        while stroke_ward.env.now < self.scenario.run_length:
            yield stroke_ward.env.timeout(self.audit_interval)
            self.audit_times.append(stroke_ward.env.now)
            self.waiting_patients_audit.append(np.mean(stroke_ward.\
                                                       waiting_patients))
            self.wait_times_audit.append(np.mean(stroke_ward.wait_times))

        stroke_ward.env.run(until=self.scenario.run_length)

        return  self.summary()

    def summary(self):
        audit_data = {
            'Audit Time': self.audit_times,
            'Mean Waiting Patients': self.waiting_patients_audit,
            'Mean Wait Time': self.wait_times_audit
        }

        summary_df = pd.DataFrame(audit_data)

        return summary_df

In [7]:
def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma

#def trace(message):
#    print(message)

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

In [8]:
# Example usage:
scenario = Scenario(num_beds=10)

warmup_auditor = WarmupAuditor(scenario, audit_interval=5)
audit_results = warmup_auditor.run_model()

print(audit_results)

<generator object WarmupAuditor.run_model at 0x12038bc30>


In this updated code:
- The admit_patient function in the StrokeWard class has been updated with the provided structure.
- The WarmupAuditor class has been added to periodically collect information on the number of patients waiting for a bed and waiting times at specified time intervals.
- The WarmupAuditor class has a run_model method that runs the StrokeWard model and collects audit data at specified intervals.
The summary method in the WarmupAuditor class returns a summary DataFrame with the mean number of patients waiting for a bed and mean waiting times at each audit period.
- The example usage demonstrates creating a WarmupAuditor instance, running the model, and printing the audit results.

With these changes, the model now includes the provided structure for the admit_patient function and the WarmupAuditor class to collect and summarize warmup data during the simulation run.

**Reflection**

The admit_patients function was passed back to the LLM to prevent it from using the incorrect implementation in the future. Because of this, it also created its own trace function, which did not include an option to turn this off. It attempted to create a WarmupAuditor class that collected the required data. However, the run model part of the function is not separate to the auditor so it sets the model to run after it tries to collect data. Therefore, it is returning a generator object and is not implementing the auditor correctly.

**Asking the LLM to fix**

**Prompt**

This returns a generator function and not a dataframe. There should be a separate run_model method that uses env.process(self.audit_model()) and audit_model method that uses env.timeout to collect the model data.

**LLM Output**

To address the issue of the run_model method returning a generator function instead of a DataFrame, and to separate the run_model and audit_model methods, you can modify the WarmupAuditor class as follows:

In [9]:
class WarmupAuditor:
    def __init__(self, scenario, audit_interval):
        self.scenario = scenario
        self.audit_interval = audit_interval
        self.audit_times = []
        self.waiting_patients_audit = []
        self.wait_times_audit = []

    def run_model(self):
        stroke_ward = StrokeWard(self.scenario)
        env = stroke_ward.env

        PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
        for patient_type in PATIENTS:
            env.process(stroke_ward.patient_arrival(patient_type))

        env.process(self.audit_model(stroke_ward))
        env.run(until=self.scenario.run_length)

        return self.summary()

    def audit_model(self, stroke_ward):
        while stroke_ward.env.now < self.scenario.run_length:
            yield stroke_ward.env.timeout(self.audit_interval)
            self.audit_times.append(stroke_ward.env.now)
            self.waiting_patients_audit.append(np.mean(stroke_ward.\
                                                       waiting_patients))
            self.wait_times_audit.append(np.mean(stroke_ward.wait_times))

    def summary(self):
        audit_data = {
            'Audit Time': self.audit_times,
            'Mean Waiting Patients': self.waiting_patients_audit,
            'Mean Wait Time': self.wait_times_audit
        }

        summary_df = pd.DataFrame(audit_data)

        return summary_df

In this updated WarmupAuditor class:
- The run_model method now creates an instance of StrokeWard and retrieves the simulation environment (env).
- The run_model method then starts the patient arrival processes and the audit_model process using env.process.
- The audit_model method is a separate generator function that uses env.timeout to collect audit data at specified intervals.
- The run_model method runs the simulation until the specified run length and returns the summary DataFrame.

With these changes, the run_model method no longer returns a generator function, and the audit_model method is responsible for collecting audit data during the simulation run using env.timeout. The run_model method starts the simulation and the audit process, and returns the summary DataFrame after the simulation is complete.

**Additional code to run**

In [10]:
scenario = Scenario(num_beds=10, seed=42) ### MODIFICATION - added seed ###

warmup_auditor = WarmupAuditor(scenario, audit_interval=5)
audit_results = warmup_auditor.run_model()

print(audit_results)

    Audit Time  Mean Waiting Patients  Mean Wait Time
0            5               0.000000        0.000000
1           10               0.000000        0.000000
2           15               0.000000        0.000000
3           20               0.000000        0.000000
4           25               0.000000        0.000000
5           30               0.000000        0.000000
6           35               0.000000        0.000000
7           40               0.017857        0.016250
8           45               0.031250        0.045000
9           50               0.773333        0.344667
10          55               1.144578        0.621928
11          60               1.272727        0.767841
12          65               1.602041        0.975000
13          70               1.666667        1.095370
14          75               1.637931        1.077500
15          80               1.557377        1.028607
16          85               1.648855        1.039237
17          90              

**Checks**

In [11]:
# Changing audit time
scenario = Scenario(num_beds=10, seed=42)

warmup_auditor = WarmupAuditor(scenario, audit_interval=10)
results = warmup_auditor.run_model()

results

,Audit Time,Mean Waiting Patients,Mean Wait Time
0,10,0.000000,0.000000
1,20,0.000000,0.000000
2,30,0.000000,0.000000
3,40,0.017857,0.016250
4,50,0.773333,0.344667
5,60,1.272727,0.767841
6,70,1.666667,1.095370
7,80,1.557377,1.028607
8,90,1.608392,1.039231
9,100,1.522293,0.986306


In [12]:
# Changing run time
scenario = Scenario(num_beds=10, run_length=50, seed=42)

warmup_auditor = WarmupAuditor(scenario, audit_interval=5)
results = warmup_auditor.run_model()

results

,Audit Time,Mean Waiting Patients,Mean Wait Time
0,5,0.000000,0.00000
1,10,0.000000,0.00000
2,15,0.000000,0.00000
3,20,0.000000,0.00000
4,25,0.000000,0.00000
5,30,0.000000,0.00000
6,35,0.000000,0.00000
7,40,0.017857,0.01625
8,45,0.031250,0.04500


In [13]:
# Changing number of beds
scenario = Scenario(num_beds=1, seed=42)

warmup_auditor = WarmupAuditor(scenario, audit_interval=5)
results = warmup_auditor.run_model()

results

,Audit Time,Mean Waiting Patients,Mean Wait Time
0,5,0.000000,0.000000
1,10,7.000000,4.515000
2,15,7.800000,5.428000
3,20,10.333333,7.308333
4,25,12.714286,8.990000
5,30,19.800000,13.273000
6,35,19.800000,13.273000
7,40,22.000000,14.840909
8,45,26.538462,18.509231
9,50,30.400000,21.306667


In [15]:
# Using no seeds
scenario = Scenario(num_beds=10)

warmup_auditor = WarmupAuditor(scenario, audit_interval=5)
results = warmup_auditor.run_model()

results

,Audit Time,Mean Waiting Patients,Mean Wait Time
0,5,0.000000,0.000000
1,10,0.000000,0.000000
2,15,0.000000,0.000000
3,20,0.031250,0.086875
4,25,0.022727,0.064091
5,30,0.057692,0.077308
6,35,0.046875,0.070313
7,40,0.042254,0.063380
8,45,0.038961,0.058442
9,50,0.036585,0.054878


**Reflection**

With additional prompting, the LLM has been able to create an output summary dataframe containing audit data. This did require a clear prompt of exactly what the output should be. The summary statistics are different each time the code is run unless a random seed is added. The audit period can be changed to collect data at different intervals. Changing the run length changes the period over which the model is run and changing the number of beds changed the waiting times and number of waiting patients as expected.